In [3]:
import pandas as pd

In [8]:
df = pd.read_csv('./processed_data/filtered_m.csv')

In [9]:
df

,bbl,10_bldgclass,10_lotarea,10_bldgarea,10_histdist,10_landmark,10_ownername,10_xcoord,10_ycoord,11_bldgclass,...,20_histdist,20_landmark,20_ownername,20_xcoord,20_ycoord,cd,council,address,xcoord,ycoord
0,1000090007,M1,6037.0,8685.0,,JAMES WATSON HOUSE,SAVANNAH TEACHERS PRO,0980452,0195261,M1,...,NaN,INDIVIDUAL LANDMARK,THE MISSION OF OUR LADY OF ROSARY OF THE CITY OF,9.804570e+05,195256.000000,101.0,1.0,7 STATE STREET,9.804570e+05,195256.000000
1,1000410022,M1,6003.0,24000.0,,,CH OUR LADY VICTORY,0981832,0196858,M1,...,NaN,NaN,CH OUR LADY VICTORY,9.818320e+05,196858.000000,101.0,1.0,56 WILLIAM STREET,9.818320e+05,196858.000000
2,1000490001,M1,98432.0,15928.0,,TRINITY CHURCH AND GRAVEYARD,PARISH OF TRINITY CHR,0980911,0197275,M1,...,NaN,INDIVIDUAL LANDMARK,PARISH OF TRINITY CHRH,9.809070e+05,197274.000000,101.0,1.0,75 BROADWAY,9.809070e+05,197274.000000
3,1000540001,O4,66222.0,1415086.0,,,LOWER MANHATTAN DEVE,0980565,0197885,O4,...,NaN,NaN,LOWER MANHATTAN DEVE,9.805380e+05,197809.000000,101.0,1.0,130 GREENWICH STREET,9.805380e+05,197809.000000
4,1000670030,M1,3423.0,3144.0,,JOHN STREET METHODIST CHURCH,BBV US REAL ESTATE FU,0981992,0197621,M1,...,NaN,INDIVIDUAL LANDMARK,JOHN STREET UNITED METHODIST CHURCH,9.819910e+05,197619.000000,101.0,1.0,44 JOHN STREET,9.819910e+05,197619.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1020940032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,THE TRUSTEES OF COLUMBIA UNIVERSITY IN T HE CI...,9.981730e+05,240947.000000,109.0,7.0,3581 BROADWAY,9.981730e+05,240947.000000
1021,1014360113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,REDEMPTORIST FATHERS OF NEW YORK,9.948320e+05,216685.000000,108.0,5.0,321 EAST 61 STREET,9.948320e+05,216685.000000
1022,1019239052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1871 PARK EQUITIES LLC,NaN,NaN,110.0,9.0,WEST 118 STREET,NaN,NaN
1023,1000540040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,PORT AUTHORITY OF NY AND NJ,9.805860e+05,198006.000000,101.0,1.0,130 LIBERTY STREET,9.805860e+05,198006.000000


In [10]:
from collections import defaultdict 
#find counts for each year
bldgclass_cols = [col for col in list(df) if 'bldgclass' in col]
m_counts = defaultdict(int)
for col in bldgclass_cols:
    m_counts[col] = len(df[df[col].str.startswith('M', na=False)])

In [11]:
pd.DataFrame.from_dict(m_counts , orient='index')

,0
10_bldgclass,976
11_bldgclass,974
12_bldgclass,977
13_bldgclass,967
14_bldgclass,960
15_bldgclass,956
16_bldgclass,946
17_bldgclass,937
18_bldgclass,914
19_bldgclass,910


In [12]:
#get entries where RF has been changed
bldgclass_cols = [col for col in list(df) if 'bldgclass' in col]

In [33]:
changed = []

for i, row in df.iterrows():
    for year in range(10,20):
        #compare if M changed to anything else
        if str(row[f'{year}_bldgclass']).startswith('M') and not str(row[f'{year + 1}_bldgclass']).startswith('M'):
            try:
                changed.append((row['bbl'], 
                                str(row[f'{year}_bldgclass']),
                                str(row[f'{year+ 1}_bldgclass']),
                                f'20{year} to 20{year+1}',
                                float(row[f'{year}_xcoord']),
                                float(row[f'{year}_ycoord']), 'Removed'))
            except:
                changed.append((row['bbl'], 
                                str(row[f'{year}_bldgclass']),
                                str(row[f'{year+ 1}_bldgclass']),
                                f'20{year} to 20{year+1}',
                                float(row[f'{year -1}_xcoord']),
                                float(row[f'{year -1}_ycoord']), 'Removed'))
        if not str(row[f'{year}_bldgclass']).startswith('M') and str(row[f'{year + 1}_bldgclass']).startswith('M'):
            try:
                changed.append((row['bbl'], 
                                str(row[f'{year}_bldgclass']),
                                str(row[f'{year+ 1}_bldgclass']),
                                f'20{year} to 20{year+1}',
                                float(row[f'{year}_xcoord']),
                                float(row[f'{year}_ycoord']), 'Added'))
            except:
                changed.append((row['bbl'], 
                                str(row[f'{year}_bldgclass']),
                                str(row[f'{year+ 1}_bldgclass']),
                                f'20{year} to 20{year+1}',
                                float(row[f'{year -1}_xcoord']),
                                float(row[f'{year -1}_ycoord']), 'Added'))

In [34]:
changed_df = pd.DataFrame(changed, columns= ['bbl','bldguse_1', 'bldguse_2', 'years', 'X', 'Y','type'])
changed_df.to_csv('./processed_data/changed.csv', index= False)
changed_df.head()

,bbl,bldguse_1,bldguse_2,years,X,Y,type
0,1000540001,Z9,M1,2016 to 2017,980563.0,197886.0,Added
1,1000540001,M1,O2,2017 to 2018,980563.0,197886.0,Removed
2,1000870001,Z8,M1,2017 to 2018,981594.0,198484.0,Added
3,1000970051,M1,W3,2012 to 2013,983567.0,197149.0,Removed
4,1001960027,K4,M1,2019 to 2020,983862.0,200985.0,Added


In [42]:
changed_df[changed_df['type'] == 'Removed'].groupby(['years','type'])['bbl'].count()

years         type   
2010 to 2011  Removed     5
2011 to 2012  Removed     6
2012 to 2013  Removed    19
2013 to 2014  Removed    11
2014 to 2015  Removed    10
2015 to 2016  Removed    13
2016 to 2017  Removed    14
2017 to 2018  Removed    30
2018 to 2019  Removed     6
2019 to 2020  Removed     6
Name: bbl, dtype: int64